In [1]:
import pinocchio as pin
import numpy as np
import time
import scipy
from example_robot_data import load

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

# import pycapacity 
import pycapacity as pycap

In [2]:
## from datasheet https://www.maxongroup.com/medias/sys_master/root/8882563907614/EN-21-300.pdf
# Coefficients for the motors
R_A1 = 35   # Coefficient for motor A related to pitch
R_A2 = 1.9  # Coefficient for motor A related to roll
R_B1 = 35   # Coefficient for motor B related to pitch
R_B2 = 1.9  # Coefficient for motor B related to roll

current_nominal = 3.96 # Nominal current
rendement = 0.93 * 0.8
couple_nominal = 0.167 * rendement / current_nominal

# orbita2d kinematics matrix
W = np.array([[R_A1 * R_A2, R_B1 * R_B2], [R_A1 * R_A2, -R_B1 * R_B2]])*couple_nominal
W_inv = np.linalg.inv(W)
W_inv = scipy.linalg.block_diag(W_inv,W_inv)

In [3]:
# function visualising the polutope in meshcat
def visualise_polytope(q):
    # calculate the jacobian
    pin.framesForwardKinematics(robot.model,data,q)
    pin.computeJointJacobians(robot.model,data, q)
    J = pin.getFrameJacobian(robot.model, data, robot.model.getFrameId(robot.model.frames[-1].name), pin.LOCAL_WORLD_ALIGNED)
    # use only position jacobian
    J = J[:3,:]
    
    # end-effector pose
    Xee = data.oMf[robot.model.getFrameId(robot.model.frames[-1].name)]
    
    
    # calculate the polytope
    opt = {'calculate_faces':True}
    # calculate the polytope
    for_poly = pycap.robot.force_polytope(J, 2*t_min*R_A1*R_A2*couple_nominal, 2*t_max*R_A1*R_A2*couple_nominal,options=opt)
    # meshcat triangulated mesh
    poly = g.TriangularMeshGeometry(vertices=for_poly.vertices.T/500 + Xee.translation, faces=for_poly.face_indices)
    viz.viewer['poly'].set_object(poly, g.MeshBasicMaterial(color=0x0022ff, wireframe=True, linewidth=3, opacity=0.2))
    
    # calculate the polytope
    opt = {'calculate_faces':True}
    # calculate the polytope
    for_poly = pycap.robot.force_polytope(J@W_inv.T, t_min, t_max,options=opt)
    # meshcat triangulated mesh
    poly = g.TriangularMeshGeometry(vertices=for_poly.vertices.T/500 + Xee.translation, faces=for_poly.face_indices)
    viz.viewer['poly2'].set_object(poly, g.MeshBasicMaterial(color=0xff2200, wireframe=True, linewidth=3, opacity=0.2))


    for_poly_z = pycap.robot.force_polytope(J[2,:].reshape(1,-1), 2*t_min*R_A1*R_A2*couple_nominal, 2*t_max*R_A1*R_A2*couple_nominal,options=opt)
    for_poly1_z = pycap.robot.force_polytope((J@W_inv.T)[2,:].reshape(1,-1), t_min, t_max,options=opt)
    print(f"How many kilos can Reachy carry?\nindependent axis\t{np.max(for_poly_z.vertices)/ 9.81:.2f}kg\ncoupled axis\t\t{np.max(for_poly1_z.vertices)/ 9.81:.2f}kg" )

In [4]:
# use reachy's right arm with only 4 joints for now as they are the contribute 
# to its force capacity the most
urdf_path = 'reachy.urdf'
robot = pin.RobotWrapper.BuildFromURDF(urdf_path)
model, data = robot.model, robot.data
# lock the other joints
tolock = [
    'l_shoulder_pitch',
    'l_shoulder_roll',
    'l_elbow_yaw',
    'l_elbow_pitch',
    'l_wrist_roll',
    'l_wrist_pitch',
    'l_wrist_yaw',
    'l_hand_finger',
    'l_hand_finger_mimic',
    'r_wrist_roll',
    'r_wrist_pitch',
    'r_wrist_yaw',
    'r_hand_finger',
    'r_hand_finger_mimic',
    'neck_roll',
    'neck_pitch',
    'neck_yaw',
]

# Get the ID of all existing joints
jointsToLockIDs = []
for jn in tolock:
    if model.existJointName(jn):
        jointsToLockIDs.append(model.getJointId(jn))
robot.model = pin.buildReducedModel(model, jointsToLockIDs, np.zeros(21))

In [5]:
data = robot.model.createData()

# get joint position ranges
q_max = robot.model.upperPositionLimit.T
q_min = robot.model.lowerPositionLimit.T
# get max velocity
t_max = np.ones(robot.nq)*4 # amps
t_min = -t_max

# Use robot configuration.
# q0 = np.random.uniform(q_min,q_max)
q = (q_min+q_max)/2

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.display(q)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
from ipywidgets import interact, FloatSlider
kwargs = {'q[{}]'.format(i) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i]) 
          for i,q_1 in enumerate(q)}
@interact(**kwargs)
def update(**kwargs):
    q = np.array([v  for v in kwargs.values()])
    viz.display(q)
    visualise_polytope(q)


interactive(children=(FloatSlider(value=0.0, description='q[0]', max=1.5707963267948966, min=-1.57079632679489…